::::
:::{thebe-button}
:::
::::

# Get thermal data


In [ ]:
from boilercv_docs.nbs import init

paths = init()

from boilercv_docs.nbs import HIDE
from pandas import DataFrame, read_csv
from seaborn import scatterplot

from boilercv.experiments.e230920_subcool import ALL_THERMAL_DATA

BOILING = 97.33  # (C) inferred from mean pressure over the time span
TIME = "time"
BASE_TEMP = "T0cal (C)"
SURFACE_TEMP = "T_s (C)"
WATER_TEMPS = ["Tw1cal (C)", "Tw2cal (C)", "Tw3cal (C)"]
COLS = [BASE_TEMP, SURFACE_TEMP, *WATER_TEMPS, "Q34 (W)"]

In [ ]:
data = (
    (
        DataFrame(columns=COLS)
        .assign(
            **read_csv(
                ALL_THERMAL_DATA,
                usecols=[TIME, *COLS],  # type: ignore  # pyright 1.1.348, pandas 2.1.4
                parse_dates=[TIME],
                index_col=TIME,
            )
        )
        .loc["2023-09-20":"2023-09-21", :]
    )
    .resample("s")
    .mean()
    .ffill()
    .assign(
        T_w=lambda df: df[WATER_TEMPS].mean(axis="columns"),
        superheat=lambda df: df[SURFACE_TEMP] - BOILING,
        subcool=lambda df: BOILING - df.T_w,
    )
).loc[:, ["subcool", "superheat", BASE_TEMP, SURFACE_TEMP, "Q34 (W)"]]
data

In [ ]:
ax = data[["subcool", "superheat"]].rename(columns=lambda col: col.capitalize()).plot()
ax.set_xlabel("Time")
ax.set_ylabel("ΔT (K)")

HIDE

In [ ]:
ax = data[[BASE_TEMP]].rename(columns={BASE_TEMP: "Base temperature"}).plot()
ax.set_xlabel("Time")
ax.set_ylabel("T (C)")

ax = data[[SURFACE_TEMP]].rename(columns={SURFACE_TEMP: "Surface temperature"}).plot()
ax.set_xlabel("Time")
ax.set_ylabel("T (C)")

In [ ]:
scatterplot(
    data=data.reset_index().assign(**{
        "Time (s)": lambda df: (df.time - df.time[0]).dt.total_seconds(),
        "Flux (W/cm^2)": lambda df: df["Q34 (W)"] / 0.712557392,
    }),
    x="superheat",
    y="Flux (W/cm^2)",
    hue="Time (s)",
)

HIDE